# 뉴스 요약하고 문서화하기

In [11]:
import os
from dotenv import load_dotenv

# .env 파일에 저장된 환경변수 로드
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
from openai import OpenAI

# openai api 인증 및 openai 객체 생성
client = OpenAI(api_key=OPENAI_API_KEY)

In [13]:
summary_prompt = """당신은 언어 이해 및 요약에 훈련된 고도로 숙련된 AI입니다.
다음 텍스트를 읽고 간결한 추상적 단락으로 요약했으면 합니다.
전체 텍스트를 읽을 필요 없이 토론의 요점을 이해하는 데 도움이 될 수 있는 일관되고
읽을 수 있는 요약을 제공하여 가장 중요한 요점을 유지하는 것을 목표로 합니다.
불필요한 세부 사항이나 접선 사항은 피하십시오."""
key_points_prompt = """당신은 정보를 핵심 포인트로 전달하는 데 특화된 능숙한 AI입니다.
다음 텍스트를 기반으로 논의되거나 언급된 주요 포인트를 확인하고 나열합니다.
이는 논의의 본질에 가장 중요한 아이디어, 결과 또는 주제가 되어야 합니다.
당신의 목표는 누군가가 읽을 수 있는 목록을 제공하여 이야기된 내용을 빠르게 이해하는 것입니다."""
action_items_prompt = """당신은 대화를 분석하고 행동 항목을 추출하는 데 있어 AI 전문가입니다.
본문을 검토하고 합의되거나 수행이 필요하다고 언급된 모든 작업, 과제 또는 행동을 식별하십시오.
이것들은 특정 개인에게 할당된 작업일 수도 있고 그룹이 취하기로 결정한 일반적인 행동일 수도 있습니다.
이러한 행동 항목을 명확하고 간결하게 나열하십시오."""
sentiment_prompt = """당신은 언어와 감정 분석에 전문성을 갖춘 AI로서 당신의 과제는 다음 텍스트의 감
정을 분석하는 것입니다.
토론의 전체적인 톤, 사용된 언어가 전달하는 감정, 단어와 구가 사용되는 맥락을 고려하십시오.
감정이 일반적으로 긍정적인지 부정적인지 중립적인지를 표시하고 가능한 한 당신의 분석에 대해 간략한 설명을 제
공하십시오."""

## 번역 함수

In [14]:
def text_extraction(prompt, text):
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": text}
        ],
        max_tokens=500,
        n=1,
        stop=None,
        temperature=0.5,
    )
    return response.choices[0].message['content']

## 음성 파일 -> text로 변환

In [17]:
audio_file_path = "datas/news.mp3"
with open(audio_file_path, 'rb') as audio_file:
    transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
)

## summary하기

In [20]:
abstract_summary = text_extraction(transcription, summary_prompt)
key_points = text_extraction(transcription, key_points_prompt)
action_items = text_extraction(transcription, action_items_prompt)
sentiment = text_extraction(transcription, sentiment_prompt)
news_data = {
    'abstract_summary': abstract_summary,
    'key_points': key_points,
    'action_items': action_items,
    'sentiment': sentiment
}
print(news_data)

{'abstract_summary': '최근 짧은 장마와 긴 폭염으로 인해 말벌의 활동이 증가하면서 119 소방대원의 출동이 급증하고 있습니다. 벌집 제거 작업이 활발히 이루어지고 있으며, 지난해보다 57% 증가한 4만 4천 건의 출동이 있었습니다. 소방당국은 말벌의 공격성을 고려해 주변에 벌집이 있는지 확인하고, 공격 시 20미터 거리로 빠르게 대피할 것을 권장하고 있습니다.', 'key_points': '1. **말벌의 극성**: 짧은 장마와 긴 폭염으로 인해 말벌의 활동이 증가하고 있음.\n2. **119 소방대원의 출동 증가**: 말벌 관련 신고가 급증하여 소방대원이 바빠짐.\n3. **벌집 제거 작업**: 소방대원들이 보호복을 착용하고 스프레이를 사용해 벌집을 제거하는 과정.\n4. **벌의 공격 습성**: 말벌은 어두운 색을 공격하는 경향이 있어 소방대원들은 흰색 보호복을 착용함.\n5. **출동 건수 증가**: 지난달 벌집 제거 출동 건수가 4만 4천 건으로 지난해보다 57% 증가.\n6. **주변 점검 및 대피 권장**: 소방당국은 벌집이 있는지 확인하고, 공격 시 20미터 거리로 빠르게 대피할 것을 권장.', 'action_items': '1. **벌집 제거 작업**: 119 소방대원들이 신고된 벌집을 제거하기 위해 출동하고, 보호복을 착용하여 안전하게 작업 수행.\n2. **사람 대피**: 벌집 제거 작업 전에 주변 사람들을 대피시키는 조치.\n3. **벌집 신고**: 주변에 벌집이 있는지 잘 살펴보고, 발견 시 신고할 것.\n4. **안전 거리 유지**: 벌이 공격해올 경우, 현장에서 20미터 정도 거리까지 빨리 벗어나라는 조언.\n5. **벌 활동 모니터링**: 8월과 9월 동안 벌의 활동을 주의 깊게 관찰하고, 쏘임사고를 예방하기 위한 조치 마련.', 'sentiment': '이 텍스트의 감정 분석 결과는 전반적으로 부정적입니다. \n\n### 분석 설명:\n1. **전체적인 톤**: 텍스트는 말벌의 극성과 소방대원의 바쁜 일상을 다루고

In [19]:
from openai import OpenAI
client = OpenAI()

def text_extraction(system_prompt: str, user_text: str) -> str:
    resp = client.chat.completions.create(
        model="gpt-4o-mini",  # 사용 중인 모델에 맞게 조정 가능 (gpt-4-turbo 대신 권장)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_text}
        ],
        max_tokens=500,
        temperature=0.5,
    )
    # ✅ 점 표기로 content 접근
    content = resp.choices[0].message.content
    return (content or "").strip()

audio_file_path = "datas/news.mp3"
with open(audio_file_path, "rb") as audio_file:
    t = client.audio.transcriptions.create(
        model="whisper-1",          # 또는 "gpt-4o-transcribe" 등
        file=audio_file,
        response_format="text"      # 텍스트만 받고 싶으면 OK
    )

# ✅ 전사 텍스트 확보 (객체든 문자열이든 대비)
transcription_text = getattr(t, "text", t)
if not isinstance(transcription_text, str):
    transcription_text = str(transcription_text)

# ✅ 인자 순서: (system_prompt, user_text)
abstract_summary = text_extraction(summary_prompt,   transcription_text)
key_points       = text_extraction(key_points_prompt, transcription_text)
action_items     = text_extraction(action_items_prompt, transcription_text)
sentiment        = text_extraction(sentiment_prompt,  transcription_text)

news_data = {
    "abstract_summary": abstract_summary,
    "key_points":       key_points,
    "action_items":     action_items,
    "sentiment":        sentiment
}
print(news_data)


{'abstract_summary': '짧은 장마와 긴 폭염으로 말벌의 활동이 증가하면서 119 소방대원들의 벌집 제거 작업이 활발해지고 있습니다. 최근에는 하루 최대 15건의 출동이 발생하며, 지난달에는 벌집 제거 출동이 지난해보다 57% 증가한 4만 4천 건에 달했습니다. 소방대원들은 보호복을 착용하고 스프레이를 사용해 벌집을 제거하며, 시민들에게는 벌집 발견 시 신고하고 벌이 공격할 경우 20미터 이상 거리로 대피할 것을 권고하고 있습니다.', 'key_points': '1. 짧은 장마와 긴 폭염으로 말벌의 활동이 증가하고 있음.\n2. 119 소방대원들의 출동이 잦아지고 있으며, 특히 벌집 제거 작업이 빈번함.\n3. 건물 처마 밑에 커다란 벌집이 발견되고, 수십 마리의 말벌이 활동 중.\n4. 소방대원들은 보호복을 착용하고 스프레이를 사용하여 벌집을 제거함.\n5. 벌들은 어두운 색을 공격하는 습성이 있어, 소방대원들은 하얀색 보호복을 착용함.\n6. 최근 소방 출동 건수가 급증하여 지난달 벌집 제거 출동이 4만 4천 건에 달함.\n7. 지난해 대비 벌집 제거 출동이 57% 증가함.\n8. 소방당국은 벌집 신고 및 벌의 공격 시 20미터 거리 이탈을 권장함.', 'action_items': '1. 119 소방대원들은 말벌 제거 작업에 나선다.\n2. 소방대원들은 사람들을 대피시킨 후 스프레이를 사용하여 벌집을 제거한다.\n3. 소방대원들은 벌의 공격을 피하기 위해 하얀색 보호복을 착용하고 빈틈없이 밴드를 붙인다.\n4. 소방당국은 벌집 신고를 권장하고, 벌이 공격해올 경우 20미터 거리까지 빨리 벗어날 것을 조언한다.\n5. 소방당국은 벌집 제거 출동 건수가 지난해보다 57% 증가한 사실을 인지하고 관련 조치를 강화해야 한다.', 'sentiment': '이 텍스트의 감정은 주로 중립적이며, 상황에 대한 객관적인 설명이 중심입니다. 그러나 일부 부정적인 요소도 포함되어 있습니다. \n\n전체적으로 텍스트는 말벌의 활동 증가와 그로 인한 소방대원의 바쁜

In [25]:
from docx import Document
doc = Document()

for key, value in news_data.items():
    # _로 나눈 단어들을 대문자로 만들고 공백으로 연결
    heading = ' '.join(word.capitalize() for word in key.split('_'))
    doc.add_heading(heading, level=1)
    doc.add_paragraph(value)
    # 섹션 사이에 줄바꿈 추가
    doc.add_paragraph()
    
import os
folder_path = 'output'
os.makedirs(folder_path, exist_ok=True)
# 워드 파일로 데이터 저장
doc.save('output/news_minutes.docx')